***
# Lab 2 ML7331
> ### By Allen Hoskins, Brittany Lewandowski, Alexy Morris, Fidel Tamares


***

In [1]:
#import packages
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression,SGDClassifier,LogisticRegressionCV
from sklearn.feature_selection import SelectPercentile,f_regression
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from graphviz.graphs import Digraph
from mlxtend.evaluate import paired_ttest_5x2cv
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import time
import warnings

#remove warnings after verifying code
warnings.filterwarnings("ignore")

***
# Data Preparation 1
Define and prepare your class variables. <br> 
> <font color=Blue> The team used a data set from Kaggle containing attributes about 2015 airline delays. The meaningfulness of the data is to provide both airlines and customers with information relevant to flight delays. This data is useful as it helps customers make decisions about which airline they fly with, and it helps airlines determine what the cause(s) of flight delays are. From a business perspective, knowing this information is critical as it drives an organization’s bottom line.  

> <font color=Blue> To get better insight into the attributes contained in the data set and their data types, the team ran `df.info(verbose=True,show_counts=True)` in Python. 

Use proper variable representations (int, float, one-hot, etc.).<br>

<font color=Blue> Initially starting with the three data sets they contain the following details: <br>
Flights:
> 1. The data set contains 5,819,079 entries.<br> 
> 2. The data set contains 31 columns.  <br>
> 3. There are 16 columns classified with the data type, float.<br>
> 4. There are 10 columns classified with the data type, int. <br>
> 5. There are 5 columns classified with the data type, object.

Airports:
> 1. The data set contains 322 entries.<br> 
> 2. The data set contains 7 columns.  <br>
> 3. There are 2 columns classified with the data type, float.<br>
> 4. There are 5 columns classified with the data type, object.

Airlines:
> 1. The data set contains 14 entries.<br> 
> 2. The data set contains 2 columns.  <br>
> 3. There are 2 columns classified with the data type, object. </font>

Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. <br>
> <font color=Blue>PCA was used preprocessing step to help scales the numeric predictors before fitting out selected models.

Remove variables that are not needed/useful for the analysis
> <font color=Blue>As we analyzed the initial datasets there were a lot of redudnat variables that represented the same type of information. Data points such as IATA_Code (The International Air Transport Association's) to identify an airport and the actual airports name. We also found some data points that we determined would not be important to classifying flight delays such as Flight_Number, cancellation_reason or tail_number
***

In [2]:
df_flights = pd.read_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/flights.csv',low_memory = False)

In [3]:
df_flights.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [4]:
df_airports = pd.read_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/airports.csv',low_memory = False)

In [5]:
df_airports.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB


In [6]:
df_airlines = pd.read_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/airlines.csv',low_memory = False)

In [7]:
df_airlines.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 352.0+ bytes


In [8]:
pd.set_option('display.max_columns', None)

In [9]:
#merge flight and airline information
df_merge_airline = df_flights.merge(df_airlines, left_on = 'AIRLINE', right_on = 'IATA_CODE',how='left')
df_merge_airline.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE,AIRLINE_y
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.


In [10]:
#find airports with missing LATITUDE
df_airports[df_airports['LATITUDE'].isna() == True]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [11]:
#merge origin airport information to merged data 
df_merge_origin = df_merge_airline.merge(df_airports, left_on = 'ORIGIN_AIRPORT',right_on = 'IATA_CODE', how='left')
df_merge_origin.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931


In [12]:
rename_dict_origin =  {'CITY':'CITY_ORIGIN',
                'STATE':'STATE_ORIGIN',
                'COUNTRY':'COUNTRY_ORIGIN',
                'LATITUDE':'LATITUDE_ORIGIN',
                'LONGITUDE':'LONGITUDE_ORIGIN'}
df_merge_origin.rename(columns=rename_dict_origin,inplace = True)

df_merge_origin.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931


In [13]:
#merge destination airport information to dataset
df_merge_final = df_merge_origin.merge(df_airports,left_on = 'DESTINATION_AIRPORT',right_on = 'IATA_CODE',how='left')
df_merge_final.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT_x,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,IATA_CODE,AIRPORT_y,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619


In [14]:
rename_dict_dest =  {'CITY':'CITY_DESTINATION',
                'STATE':'STATE_DESTINATION',
                'COUNTRY':'COUNTRY_DESTINATION',
                'LATITUDE':'LATITUDE_DESTINATION',
                'LONGITUDE':'LONGITUDE_DESTINATION'}
df_merge_final.rename(columns=rename_dict_dest,inplace = True)

df_merge_final.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT_x,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,IATA_CODE,AIRPORT_y,CITY_DESTINATION,STATE_DESTINATION,COUNTRY_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619


In [15]:
df_merge_final.columns= df_merge_final.columns.str.strip().str.lower()

In [16]:
df_ordered = df_merge_final.copy(deep=True)

In [17]:
col_order = ['year','month','day','day_of_week','airline_x','airline_y','flight_number','tail_number',
             'origin_airport','airport_x','city_origin','state_origin','country_origin','latitude_origin','longitude_origin',
             'destination_airport','airport_y','city_destination','state_destination','country_destination','latitude_destination','longitude_destination',
             'scheduled_departure','departure_time','departure_delay','taxi_out','wheels_off','scheduled_time','elapsed_time','air_time','distance',
             'wheels_on','taxi_in','scheduled_arrival','arrival_time','arrival_delay','diverted','cancelled','cancellation_reason','air_system_delay',
             'security_delay','airline_delay','late_aircraft_delay','weather_delay']

In [18]:
df_ordered = df_ordered[col_order]

In [19]:
rename_dict_merged =  {'airline_x':'airline_cd',
                       'airline_y':'airline_name',
                       'airport_x': 'airport_origin',
                       'airport_y':'airport_destination'}
df_ordered.rename(columns=rename_dict_merged,inplace = True)

In [21]:
df_ordered.to_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/airline_merged.csv')

***
# Data Preparation 2
Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

<font color=Blue> The final data set that we started our analysis with was named airlines_merged and contained the following details:
> 1. The data set contains 5,819,079 entries.<br> 
> 2. The data set contains 45 columns.  <br>
> 3. There are 20 columns classified with the data type, float.<br>
> 4. There are 11 columns classified with the data type, int. <br>
> 5. There are 14 columns classified with the data type, object.

The __5,819,079x45 Data Frame__ is sufficient size for us to conduct our analysis and classification of future flights. <br>
We have also included a data ditionary to better explain the purpose of each predictor. 

**Newly Created Fields:**
> Because we want to predict if an airplane is going to be delayed or not using classification models, we created a new field called `is_delay` from the `departure_delay` field. To do this, we created a function that we could pass a numeric field through and if the value for that row is greater than 0, we imputed "1" into the curated field to signifiy the flight was delayed. If the value was less than 0, we imputed "0" to signify that the flight was not delayed. 

> After creating the `is_delay` field, we then began to One Hot Encode any categorial variables so that our model could interpret and use those columns. This caused a 250k by 42 data set to become 240k by 3400.
***

In [ ]:
df = pd.read_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/airline_merged.csv',low_memory=False)

In [ ]:
df.info(verbose=True)


**Data Dictionary:**
***

| Attribute Name | Attribute Description | Non-null Count | Attribute Data Type|
| :--- | :--- | :---| :---|
| year | Year data was collected |5819079|int |
| month | Month data was collected |5819079| int |
| day | Day data was collected (1-25) |5819079| int |
| day_of_week | Day of the week data was collected (1-7) |5819079|int |
| airline_name | Name of airline ex) Alaska Airlines Inc. |5819079| object |
| airline_cd | Code for airline ex) AA |5819079|object |
| flight_number | Unique identifier for each flight |5804358| int |
| tail_number  | Unique identifier for a plane | 5819079|object |
| origin_airport | Code of origin airport ex) DFW for Dallas Fort Worth International Airport |5819079| object |
| airport_origin | Airport flight originates from ex) San Francisco International Airport  | 5332914|object  |
| city_origin | City the flight came from | 5332914|object |
| state_origin | State the flight came from | 5332914|object |
| country_origin | Country flight came from | 5332914|object |
| latitude_origin | Latitude coordinates the flight came from | 5332914|float |
| longitude_origin | Longitude coordinates the flight came from | 5332914|float |
| destination_airport | Code of destination airport ex) DFW for Dallas Fort Worth International Airport |5819079| object |
| airport_destination | Name of destination airport ex) Dallas Fort Worth International Airport |5332914|	object |
| city_destination | City of destination | 5332914|object |
| state_destination | State of destination | 5332914|object |
| country_destination | Country of destination |5332914| object |
| latitude_destination | Latitude coordinates of flight destination | 5332914|float |
| longitude_destination | Longitude coordinates of flight destination |5332914| float |
| scheduled_departure | Time of flight departure | 5819079|int |
| departure_time | Time flight departed | 5732926|float |
| departure_delay | Time flights are delayed from departure |5732926| float |
| taxi_out | Time spent taxiing to the runway  |5730032| float |
| wheels_off | Time spent removing wheels  | 5730032|float  |
| scheduled_time | When flight is scheduled to back up from gate | 5819073|float |
| elapsed_time | Calculated column derived by adding taxi & air time together |5714008| float |
| air_time | Time flight spends in the air | 5714008|float |
| distance | Distance traveled by flights | 5819079|int |
| wheels_on | Time spent |5726566| float |
| taxi_in | Time spent taxiing to the gate | 5726566|float |
| scheduled_arrival | Scheduled flight arrival time |5819079| int |
| arrival_time | Time flight actually arrived | 5726566|float |
| arrival_delay |Time flights are delayed from arrival |5714008| float |
| diverted | Binary indicator of if a flight was diverted or not (0,1) |5819079| int |
| cancelled | Binary indicator of if a flight was canceled or not (0,1) |5819079| int |
| cancellation_reason | Reason for flight cancellation | 89884|object |
| air_system_delay | Flight delays due to air systems |1063439| float |
| security_delay | Flight delays due to security issues |1063439| float |
| airline_delay | Flight delays due to an airline specific problem |1063439| float |
| late_aircraft_delay | Flight delays due to an aircraft being late | 5819079|float |
| weather_delay  | Flight delays due to weather | 1063439|float |

***

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#removing unwanted column from previous export
df = df.drop(columns =['Unnamed: 0'],axis =1)

In [ ]:
#dropping highly correlated columns
#https://www.codegrepper.com/code-examples/python/how+to+drop+highly+correlated+features
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

df = df.drop(columns = to_drop, axis=1)

In [ ]:
#selecting subset of data set
airports = ['BHM']

In [ ]:
df_air = df.loc[df['origin_airport'].isin(airports)]

In [ ]:
df_air.describe()

In [ ]:
#selecting columns that are dtype != numeric
non_numeric = df_air.select_dtypes(exclude = np.number).columns

In [ ]:
#create binary response for logistic regression
# create a function
def is_delayed(delay,axis=1):
    if delay <= 0:
        return 0
    else:
        return 1
    
# create a new column based on condition
df_air['is_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)

In [ ]:
df_air.info(verbose=True)

In [ ]:
#Departure Delay - response
#altered from https://www.statology.org/one-hot-encoding-in-python/

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='error',drop='first')

#perform one-hot encoding on non-numeric column s
encoder_df = pd.DataFrame(encoder.fit_transform(df_air[non_numeric]).toarray())

#return column names from encoder
encoder_df.columns = encoder.get_feature_names_out()

#merge one-hot encoded columns back with original DataFrame
encoded = df_air.join(encoder_df)

In [ ]:
#drop original columns
final_df = encoded.drop(non_numeric, axis=1)

# Modeling and Evaluations 1
**Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed).** <br>

> Two model evaluation metrics were chosen for this lab. AUC was used for Task One, and F1-score was used for Task Two.

**Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.** <br>

> The team used AUC to evaluate models built for Task One (predicting flight delays). The team believes AUC fits our data set well due to its ability to distinguish between positive and negative classes. Distinguishing between positive and negative classes is important in our project, because delays directly impact an airline’s bottom line.

> The team used F1-score to evaluate models built for Task Two (predicting flight cancellations). F1-score was chosen as an appropriate metric because we did not find recall or precision to be more important than the other. The team believes both precision and recall are important for predicting flight cancellations because we want to ensure that our model is performing accurately against both our train and test sets.

# Modeling and Evaluations 2 
Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?) <br>
> <font color=Blue> The method we decided to use to create our training and testing data sets was stratified 10-fold cross validation. This was chosen due to industry standards and therefore we found this most appropriate. This also keeps the same proprotion of the feature in interest as in the original data set.
    
> Due to extensive runtime and processing issues, for GridSearchCV, we used a 3-fold cross validation set. 
   

In [ ]:
#Specififying Stratified Kfold for cv.
skfcv = StratifiedKFold(n_splits=10)

> Creating Test and Train Split for Task 1 (Predicting Delay)

In [ ]:
#remove target variable and column target was created from training data set
target_delay_cols = ['is_delay','departure_delay']
feature_delay_cols = final_df.loc[:, ~final_df.columns.isin(target_delay_cols)].columns
X_delay = final_df[feature_delay_cols] # Features
y_delay = final_df.is_delay # Target variable

In [ ]:
#split original dataset into features and target variable
X_delay_train,X_delay_test,y_delay_train,y_delay_test=train_test_split(X_delay,y_delay,test_size=0.20,random_state=0)

In [ ]:
#imputing 0 into NaN values for logistic regression after OHE
X_delay_train = X_delay_train.fillna(0)
X_delay_test = X_delay_test.fillna(0)
y_delay_train = y_delay_train.fillna(0)
y_delay_test = y_delay_test.fillna(0)

In [ ]:
#checking shape of data
print('X_delay_train shape: ',X_delay_train.shape,'\n')
print('X_delay_test shape: ',X_delay_test.shape,'\n')
print('y_delay_train shape: ',y_delay_train.shape,'\n')
print('y_delay_test shape: ',y_delay_test.shape)

> Creating Test and Train Split for Task 2 (Predicting Cancellation)

In [ ]:
#remove target variable and column target was created from training data set
target_cancel_cols = ['cancelled','cancellation_reason']
feature_cancel_cols = final_df.loc[:, ~final_df.columns.isin(target_cancel_cols)].columns
X_cancel = final_df[feature_cancel_cols] # Features
y_cancel = final_df.is_delay # Target variable

In [ ]:
#split original dataset into features and target variable
X_cancel_train,X_cancel_test,y_cancel_train,y_cancel_test=train_test_split(X_cancel,y_cancel,test_size=0.20,random_state=0)

In [ ]:
#imputing 0 into NaN values for logistic regression after OHE
X_cancel_train = X_cancel_train.fillna(0)
X_cancel_test = X_cancel_test.fillna(0)
y_cancel_train = y_cancel_train.fillna(0)
y_cancel_test = y_cancel_test.fillna(0)

In [ ]:
#checking shape of data
print('X_cancel_train shape: ',X_cancel_train.shape,'\n')
print('X_cancel_test shape: ',X_cancel_test.shape,'\n')
print('y_cancel_train shape: ',y_cancel_train.shape,'\n')
print('y_cancel_test shape: ',y_cancel_test.shape)

# Modeling and Evaluations 3 
Create three different classification/regression models (e.g., random forest, KNN, and SVM).
> <font color=Blue> We have selected a logistic regression model to set our baseline for evaluation, random forests and KNN
    
Two modeling techniques must be new (but the third could be SVM or logistic regression).
Adjust PARAMETERS as appropriate to increase generalization performance using your chosen metric.
> <font color=Blue> The perameters we adjusted and why in each model were: <br>
> Logistic Regression: <br>
> KNN: <br>
> Random Forests: <br>

### Logistic Regression:
To simplify the code and useability of the model, we used a pipeline function from sklearn to standardize (using StandarScaler), run the data through PCA and then finally a logistic regession model using the PCA output.
***

### Logistic Regression: Task 1

In [ ]:
#Checking distribution of our response.
#The below code was altered from: https://wellsr.com/python/upsampling-and-downsampling-imbalanced-data-in-python/

print(df_air["cancelled"].value_counts())

df_air.groupby('cancelled').size().plot(kind='pie',
                                       y = "cancelld",
                                       label = "Percentage of Delays",
                                       autopct='%1.1f%%');

In [ ]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

tolerance = 0.1
l1ratio = 1
rs = 0

# Scale -> PCA -> LR 
logistic = LogisticRegression(solver = 'saga',
                                random_state = rs,
                                penalty ='elasticnet',
                                l1_ratio = l1ratio,
                                tol = tolerance)

pipe = Pipeline(steps=[("scaler", scaler),
                       ("pca", pca),
                       ("logistic", logistic)]
               )

pipe.fit(X_delay_train,y_delay_train)

In [ ]:
#predict
y_delay_pred=pipe.predict(X_delay_test)

In [ ]:
#Metrics
print("Accuracy:",metrics.accuracy_score(y_delay_test, y_delay_pred))
print("Precision:",metrics.precision_score(y_delay_test, y_delay_pred))
print("Recall:",metrics.recall_score(y_delay_test, y_delay_pred))

In [ ]:
lr_delay_matrix = confusion_matrix(y_delay_test, y_delay_pred)


lr_delay_group_names = ['True Neg','False Pos','False Neg','True Pos']

lr_delay_group_counts = ["{0:0.0f}".format(value) for value in
                lr_delay_matrix.flatten()]

lr_delay_group_percentages = ["{0:.2%}".format(value) for value in
                     lr_delay_matrix.flatten()/np.sum(lr_delay_matrix)]

lr_delay_labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(lr_delay_group_names,lr_delay_group_counts,lr_delay_group_percentages)]

lr_delay_labels = np.asarray(lr_delay_labels).reshape(2,2)

sns.heatmap(lr_delay_matrix, annot=lr_delay_labels, fmt='', cmap='Blues');

### Logistic Regression: Task 2

In [ ]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

tolerance = 0.1
l1ratio = 1
rs = 0

# Scale -> PCA -> LR 
logistic = LogisticRegression(solver = 'saga',
                                random_state = rs,
                                penalty ='elasticnet',
                                l1_ratio = l1ratio,
                                tol = tolerance)

pipe = Pipeline(steps=[("scaler", scaler),
                       ("pca", pca),
                       ("logistic", logistic)]
               )

pipe.fit(X_cancel_train,y_cancel_train)

In [ ]:
#predict
y_cancel_pred=pipe.predict(X_cancel_test)

In [ ]:
#Metrics
print("Accuracy:",metrics.accuracy_score(y_cancel_test, y_cancel_pred))
print("Precision:",metrics.precision_score(y_cancel_test, y_cancel_pred))
print("Recall:",metrics.recall_score(y_cancel_test, y_cancel_pred))

In [ ]:
lr_cancel_matrix = confusion_matrix(y_cancel_test, y_cancel_pred)


lr_cancel_group_names = ['True Neg','False Pos','False Neg','True Pos']

lr_cancel_group_counts = ["{0:0.0f}".format(value) for value in
                lr_cancel_matrix.flatten()]

lr_cancel_group_percentages = ["{0:.2%}".format(value) for value in
                     lr_cancel_matrix.flatten()/np.sum(lr_cancel_matrix)]

lr_cancel_labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(lr_cancel_group_names,lr_cancel_group_counts,lr_cancel_group_percentages)]

lr_cancel_labels = np.asarray(lr_cancel_labels).reshape(2,2)

sns.heatmap(lr_cancel_matrix, annot=lr_cancel_labels, fmt='', cmap='Blues');




In [ ]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a
#https://lightgbm.readthedocs.io/en/latest/Parameters.html#objective

# Define a pipeline to search for the best combination of PCA truncation

pca_lgbm = PCA(n_components = .95,
               svd_solver = 'auto')


In [ ]:
# Define a Standard Scaler to normalize inputs

scaler_lgbm = RobustScaler()

lr = 0.15
boost = 'dart' #'gbdt' 
obj = 'binary'
tl = 'voting' # 'data'
numiter = 1000
mbin = 300
nleaves = 42


In [ ]:
# Scale -> PCA -> LR 
lgbm = lgb.LGBMClassifier(learning_rate = lr,
                          boosting = boost,
                          objective = obj,
                          tree_learner = tl,
                          #num_iterations = numiter,
                          num_leaves = nleaves,
                          max_bin = mbin,
                          extra_trees = True)


In [ ]:
%%time
#start_lgbm = time.time()
pipe_lgbm = Pipeline(steps=[("scaler", scaler_lgbm),
                       ("pca", pca_lgbm),
                       ("LightGBM", lgbm)])

pipe_lgbm.fit(X_train,y_train)
#end_lgbm = time.time()

In [ ]:
#predict
y_pred_lgbm=pipe_lgbm.predict(X_test)

In [ ]:
accuracy_lgbm = metrics.accuracy_score(y_test,y_pred_lgbm)
precision_lgbm = metrics.precision_score(y_test, y_pred_lgbm)
recall_lgbm = metrics.recall_score(y_test, y_pred_lgbm)

In [ ]:
#print results 
print('____________Start LightGBM Base Model Results______________:')
print('Accuracy:\n',accuracy_lgbm,'\n')
print('Precision:\n',recall_lgbm,'\n')
print('Recall:\n',recall_lgbm,'\n')
print('____________End LightGBM Base Model Results________________')

***
## Classification Model #2
### Random Forest:
***

### Random Forest: Task 1

In [ ]:
#Creating our random forest pipeline.
rf = RandomForestClassifier(n_estimators=100, random_state=0)

In [ ]:
#Fitting random forest model on training data.
rf.fit(X_delay_train, y_delay_train)

In [ ]:
#Predicting on our test set.
y_pred_rf = rf.predict(X_delay_test)

In [ ]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
rfc_cv_score = cross_val_score(rf, X_delay_test, y_delay_test, cv=skfcv, scoring='accuracy')


In [ ]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
print("=== Confusion Matrix ===")
print(confusion_matrix(y_delay_test, y_pred_rf))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_delay_test, y_pred_rf))
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

### Random Forest Grid Search: Task 1

In [ ]:
#Task 1 Grid Search parameters: 
#Specifying the number of trees in random forest:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)]
#Specifying criterion for each split:
criterion = ['gini','entropy']
#Specifying the number of features to consider at every split:
max_features = ['auto','sqrt', 'log2','none']

In [ ]:
#Creating pipeline for GridSearch:
#Creating and printing my random grid with parameters defined above:
param_grid = {'n_estimators': n_estimators,
               'criterion': criterion,
               'max_features': max_features}
print(param_grid)

In [ ]:
#Base RF model used to tune:
rf = RandomForestClassifier(random_state=0)

In [ ]:
#Throwing RF through a GridSearch:
rf_random = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, verbose=1)

In [ ]:
#Fiting the random search model on our training data:
rf_random.fit(X_delay_train, y_delay_train)

In [ ]:
#Predict CV data from training data:
rf_random_pred.fit(X_delay_train, y_delay_train)

In [ ]:
#Viewing the best parameters selected from GridSearch:
rf_random.best_params_

### Re-Fitting Random Forest with GridSearch Parameters: Task 1

In [ ]:
#Defining RF model with best parameters from GridSearch:
rf_gs = RandomForestClassifier(criterion='entropy',max_features='auto',n_estimators=100,random_state=0) #INSERT IN BEST GRIDSEARCH PARAMS

In [ ]:
#Re-fitting RF model with GS parameters and SKFCV on training data:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
rfc_cv_score_train = cross_val_score(rf_gs, X_delay_train, y_delay_train, cv=skfcv, scoring='accuracy')

rf_gs.fit(X_delay_train,y_delay_train)
rf_gs_pred = rf_gs.predict(X_delay_test)

In [ ]:
#Viewing performance metrics:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
print("=== Confusion Matrix ===")
print(confusion_matrix(y_delay_test, rf_gs_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_delay_test, rf_gs_pred))
print("=== All AUC Scores ===")
print(rfc_cv_score_train)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score_train.mean())

### Feature Importances for GridSearch RF Model: Task 1

In [ ]:
#Fiting the random search model on our training data:
rf_gs.fit(X_delay_train, y_delay_train)
importances = rf_gs.feature_importances_

In [ ]:
# Getting indices of N = 3 maximum values
x = np.argsort(importances)[::-1][:5]
print("Indices:",x)

# Getting N maximum values
print("Values:",importances[x])

In [ ]:
plt.title('RF Train Data Feature Importances')
(pd.Series(rf_gs.feature_importances_, index=X_delay.columns)
   .nlargest(5)
   .plot(kind='barh')) ;

### Visualizing task one RF GridSearch Model: Task 1

In [ ]:
#ROC Curve
basic_ypred_proba = rf_gs_test.predict_proba(X_delay_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_delay_test,  basic_ypred_proba)
basic_lr_auc = metrics.roc_auc_score(y_delay_test, basic_ypred_proba)
plt.plot(fpr,tpr,label="Test RF with CV, auc= "+str(round(basic_lr_auc,5)))
plt.legend(loc=0)
plt.show();

### Base Random Forest: Task 2

In [ ]:
#Creating our random forest pipeline.
rf = RandomForestClassifier(random_state=0)

In [ ]:
#Fitting random forest model on training data.
rf.fit(X_cancel_train, y_cancel_train)

In [ ]:
#Predicting on our test set.
y_pred_rf = rf.predict(X_cancel_test)

In [ ]:
#Running RF model through CV:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
rfc_cv_score2 = cross_val_score(rf, X_cancel_test, y_cancel_test, cv=skfcv, scoring='accuracy')

In [ ]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
print("=== Confusion Matrix ===")
print(confusion_matrix(y_cancel_test, y_pred_rf))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_cancel_test, y_pred_rf))
print("=== All AUC Scores ===")
print(rfc_cv_score2)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score2.mean())

### RF Grid Search: Task 2

In [ ]:
#Creating pipeline for GridSearch:
#Creating and printing my random grid with parameters defined above:
param_grid2 = {'n_estimators': n_estimators,
               'criterion': criterion,
               'max_features': max_features}
print(param_grid2)

In [ ]:
#Base RF model used to tune:
rf = RandomForestClassifier(random_state=0)

In [ ]:
#Throwing RF through a GridSearch:
rf_random2 = GridSearchCV(estimator = rf, param_grid = param_grid2, cv = 3, verbose=1)

In [ ]:
#Fiting the random search model on our training data:
rf_random2.fit(X_cancel_train, y_cancel_train)

In [ ]:
#Viewing the best parameters selected from GridSearch:
rf_random2.best_params_

### Re-Fitting Random Forest with GridSearch Parameters: Task 1

In [ ]:
#Defining RF model with best parameters from GridSearch:
rf_gs2 = RandomForestClassifier(criterion='gini',max_features='auto',n_estimators=144,random_state=0) #INSERT IN BEST GRIDSEARCH PARAMS

In [ ]:
#fit and predict on refitted RF with new Parameters
rf_gs2.fit(X_cancel_train,y_cancel_train)
rf_gs2_pred = rf_gs2.predict(X_cancel_test)

In [ ]:
#Re-fitting RF model with GS parameters and SKFCV:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
rfc_cv_score_train_2 = cross_val_score(rf_gs2, X_cancel_train, y_cancel_train, cv=skfcv, scoring='accuracy')

In [ ]:
#Viewing performance metrics:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
print("=== Confusion Matrix ===")
print(confusion_matrix(y_delay_test, rf_gs2_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_delay_test, rf_gs2_pred))
print("=== All AUC Scores ===")
print(rfc_cv_score_train_2)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score_train_2.mean())

### Feature Importances for GridSearch RF Model: Task 2

In [ ]:
#Fiting the random search model on our training data:
importances2 = rf_gs2.feature_importances_

In [ ]:
#Getting indices of N=5 maximum values:
x=np.argsort(importances2)[::-1][:5]
print("Indices:",x)


#Getting 5 maximum values:
print("Values:", importances2[x])

In [ ]:
plt.title('RF Train Data Feature Importances')
(pd.Series(rf_gs2.feature_importances_, index=X_cancel.columns)
   .nlargest(5)
.plot(kind='barh'))

In [ ]:
plt.title('RF Train Data Feature Importances')
(pd.Series(rf_gs2.feature_importances_, index=X_cancel.columns)
   .nsmallest(5)
.plot(kind='barh'))

### Visualizing task one RF GridSearch Model: Task 2

In [ ]:
#ROC Curve
basic_ypred_proba = rf_gs2.predict_proba(X_cancel_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_delay_test,  basic_ypred_proba)
basic_lr_auc = metrics.roc_auc_score(y_cancel_test, basic_ypred_proba)
plt.plot(fpr,tpr,label="Test RF with CV, auc= "+str(round(basic_lr_auc,5)))
plt.legend(loc=0)
plt.show();

***
## Classification Model #2
### KNN
***
### KNN: Task 1

In [ ]:
scaler_knn = RobustScaler()

In [ ]:
knn_delay = KNeighborsClassifier(n_neighbors = 17, weights='distance')

pipe_knn_delay = Pipeline(steps=[("scaler", scaler_knn),
                       ("KNN", knn_delay)])

pipe_knn_delay.fit(X_delay_train,y_delay_train)

y_pred_pipe_knn_delay = pipe_knn_delay.predict(X_delay_test)

pipe_knn_cv_score_delay = cross_val_score(pipe_knn_delay, X_delay_test, y_delay_test, cv=skfcv, scoring='accuracy')

In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_delay_test, y_pred_pipe_knn_delay))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_delay_test, y_pred_pipe_knn_delay))
print("=== All AUC Scores ===")
print(pipe_knn3_cv_score_cancel)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ", pipe_knn_cv_score_delay.mean())

### KNN: Task 2

In [ ]:
knn_cancel = KNeighborsClassifier(n_neighbors = 17, weights='distance')

pipe_knn_cancel = Pipeline(steps=[("scaler", scaler_knn),
                       ("KNN", knn_cancel)])

pipe_knn_cancel.fit(X_cancel_train,y_cancel_train)

y_pred_pipe_knn_cancel = pipe_knn_cancel.predict(X_cancel_test)

pipe_knn_cv_score = cross_val_score(pipe_knn_cancel, X_cancel_test, y_cancel_test, cv=skfcv, scoring='accuracy')


In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_canel_test, y_pred_pipe_knn_cancel))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_cancel_test, y_pred_pipe_knn_cancel))
print("=== All AUC Scores ===")
print(pipe_knn_cv_score_cancel)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ", pipe_knn_cv_score_cancel.mean())

# Modeling and Evaluations 4
Analyze the results using your chosen method of evaluation. Use 
visualizations of the results to bolster the analysis. Explain any visuals and analyze why 
they are interesting to someone that might use this model

### Task 1 Evaluation:

### Task 2 Evaluation:

# Modeling and Evaluations 5
Discuss the advantages of each model for each classification task, if any. If 
there are not advantages, explain why. Is any model better than another? Is the 
difference significant with 95% confidence? Use proper statistical comparison methods

In [ ]:
#http://rasbt.github.io/mlxtend/user_guide/evaluate/paired_ttest_5x2cv/
t, p = paired_ttest_5x2cv(estimator1=rf_gs_test,
                          estimator2=pipe_lgbm,
                          X=X_delay_train, y=y_delay_train,
                          random_seed=1)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

In [ ]:
#http://rasbt.github.io/mlxtend/user_guide/evaluate/paired_ttest_5x2cv/
t, p = paired_ttest_5x2cv(estimator1=rf_gs_test,
                          estimator2=pipe_lgbm,
                          X=X_cancel_train, y=y_cancel_train,
                          random_seed=1)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

# Modeling and Evaluations 6
Which attributes from your analysis are most important? Use proper 
methods discussed in class to evaluate the importance of different attributes. Discuss 
the results and hypothesize about why certain attributes are more important than others 
for a given classification task

***
# Deployment
**How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)?** <br>
 > Our model would be useful for travelers and airlines. For travellers, having a flight delayed or cancelled can impact their travel plans, especially if there are connecting flights. Being able to identify airlines with higher rates of delays and cancellations can help them make better decisions. As an airline, this could be useful in identifying the major indicators in the different types of delays. For every airline flight cancelled or delayed is money that cannot be recouped and can result in additional paid compensation (lost luggage/ lodging expenses/ food vouchers) while still having to resolve the issue with the customer. <br>
 
 
**How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties?** <br>
> We would measure the models value as a cost savings to both travelers and airlines. This would be a simple model to deploy and since it uses flight records. Maintaining a database that appends new records could provide timely data for both parties in avoiding travel disruptions and providing flags that may add additional costs to each flight. <br>


**What other data should be collected?**<br>
> Other datapoints that could be useful to add value to the model would be: <br>


| # | Feature Addition | Description | Use Case |
| :--- | :--- | :--- |:--- |
| 1 | Aircraft Model | Plane Models  | This could be used to better understand flight efficiency based on age of and plane sophistication |
| 2 | Maintenance History | Last maintenance check, frequency of tune ups, number of major overhauls | Maintenance history could help determine how fast a plane gets in the air, the speeds it can reach and the turn around from one flight to another |
| 3 | Customer Satisfaction Score | We imagine a survey score on different categories like comfort, entertainment, etc | Customer satisfaction scores could be used to target high priority issues the resolve. An example would be If customers rated the flight with a low timeliness score, turn-around procedures would be identified and reviewed |
| 4 | Pilot Years of Experience | Measured in years or months,  could be categorized by position title  | Our assumption is pilot years of experience may influence how long it takes to get planes off the ground and how they handle weather or flight routes |
| 5 | Flight Costs | Measured in Dollars | This could be a measure for airlines that have little to no delay. Our assumption is punctuality and timeliness could influence consumer trust in an airline, the higher the demand would then in turn result in a higher price. Finding that influences flight costs may be correlated with flight efficiency |
| 6 | Flight Weight | Numeric: in lbs or kg  | Our assumption is a heavier plane takes longer to load and could influence the flight speed which would influence departure and arrival delays  |

<br>**How often would the model need to be updated, etc.?** <br>

> The model would not need major updates unless that is a signiciant change to the industry through regulation or technoloical changes. The model is based on past records that share similar economic and political environments as well as analyzing seasonal travel behavior. If these environments do not change the model should be resilient as time goes on.


***
# Exceptional Work
• You have free reign to provide additional modeling. 
• One idea: grid search parameters in a parallelized fashion and visualize the 
performances across attributes. Which parameters are most significant for making a 
good model for each classification algorithm
***

### Catboost

In [ ]:
df_air_catb = df_air.copy()

In [ ]:
#convert object fields and category fields 
categories = ['year','month','day','day_of_week','airline_cd','airline_name','flight_number','origin_airport','tail_number',
              'airport_origin','city_origin','state_origin','country_origin','destination_airport','airport_destination',
              'city_destination','state_destination','country_destination','cancellation_reason','is_delay','cancelled']

df_air_catb[categories] = df_air_catb[categories].fillna('Unknown')

In [ ]:
df_air_catb[categories] = df_air_catb[categories].astype('category')

In [ ]:
df_air_catb.info(verbose = True)

In [ ]:
#remove target variable and column target was created from training data set
target_cat_delay_cols = ['is_delay','departure_delay']
feature_cat_delay_cols = df_air.loc[:, ~df_air_catb.columns.isin(target_cat_delay_cols)].columns
X_cat_delay = df_air_catb[feature_cat_delay_cols] # Features
y_cat_delay = df_air_catb.is_delay # Target variable

In [ ]:
#remove target variable and column target was created from training data set
target_cat_cancel_cols = ['cancelled','cancellation_reason','is_delay']
feature_cat_cancel_cols = df_air.loc[:, ~df_air_catb.columns.isin(target_cat_cancel_cols)].columns
X_cat_cancel = df_air_catb[feature_cat_cancel_cols] # Features
y_cat_cancel = df_air_catb.cancelled # Target variable

In [ ]:
#split original dataset into features and target variable
X_cat_delay_train,X_cat_delay_test,y_cat_delay_train,y_cat_delay_test=train_test_split(X_cat_delay,y_cat_delay,test_size=0.20,random_state=0)

In [ ]:
#split original dataset into features and target variable
X_cat_cancel_train,X_cat_cancel_test,y_cat_cancel_train,y_cat_cancel_test=train_test_split(X_cat_cancel,y_cat_cancel,test_size=0.20,random_state=0)

In [ ]:
#checking shape of data
print('X_cat_delay_train shape: ',X_cat_delay_train.shape,'\n')
print('X_cat_delay_test shape: ',X_cat_delay_test.shape,'\n')
print('y_cat_delay_train shape: ',y_cat_delay_train.shape,'\n')
print('y_cat_delay_test shape: ',y_cat_delay_test.shape)

In [ ]:
#checking shape of data
print('X_cat_cancel_train shape: ',X_cat_cancel_train.shape,'\n')
print('X_cat_cancel_test shape: ',X_cat_cancel_test.shape,'\n')
print('y_cat_cancel_train shape: ',y_cat_cancel_train.shape,'\n')
print('y_cat_cancel_test shape: ',y_cat_cancel_test.shape)

In [ ]:
cat_feat_delay = df_air_catb.select_dtypes(include=['category']).columns.tolist()
cat_feat_cancel = df_air_catb.select_dtypes(include=['category']).columns.tolist()

In [ ]:
#remove response variable
cat_feat_delay.remove('is_delay')
cat_feat_cancel.remove('cancelled')
cat_feat_cancel.remove('cancellation_reason')
cat_feat_cancel.remove('is_delay')

In [ ]:
%%time
#CatBoost for Is_delay
#https://www.kaggle.com/code/mitribunskiy/tutorial-catboost-overview/notebook

seed = 0
params = {'loss_function':'Logloss', # objective function
          'custom_metric':['AUC','Accuracy','F1'], # metrics
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': seed,
          'learning_rate': .25,
          'cat_features':cat_feat_delay,
          'grow_policy': 'Lossguide',
          'thread_count': -1,
          'early_stopping_rounds':200
         }

cbc_delay = CatBoostClassifier(**params)
cbc_delay.fit(X_cat_delay_train, y_cat_delay_train, # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
          eval_set=(X_cat_delay_test, y_cat_delay_test), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         );

In [ ]:
cbc_delay.plot_tree(tree_idx=298)

In [ ]:

#get feature importance from Catboost model
feature_importance = cbc_delay.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_cat_delay_test.columns)[sorted_idx])
plt.title('Catboost Feature Importance: Delay')

In [ ]:
%%time
#CatBoost for Cancellation


#https://www.kaggle.com/code/mitribunskiy/tutorial-catboost-overview/notebook

seed = 0
params = {'loss_function':'Logloss', # objective function
          'custom_metric':['AUC','Accuracy','F1'], # metrics
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': seed,
          'learning_rate': .03,
          'cat_features':cat_feat_cancel,
          'thread_count': -1,
          'early_stopping_rounds':10
         }

cbc_cancel = CatBoostClassifier(**params)
cbc_cancel.fit(X_cat_cancel_train, y_cat_cancel_train, # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
          eval_set=(X_cat_cancel_test, y_cat_cancel_test), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         );

In [ ]:
import matplotlib.pyplot as plt
#get feature importance from Catboost model
feature_importance = cbc_cancel.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_cat_cancel_test.columns)[sorted_idx])
plt.title('Catboost Feature Importance: Cancellations')

In [ ]:
cbc_cancel.plot_tree(tree_idx=607)